In [2]:
from peaceful_pie.unity_comms import UnityComms
import argparse
import gym
from gym import Env
from gym.spaces import Box, MultiBinary,Discrete
import numpy as np
import os
from torch.utils.tensorboard import SummaryWriter
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import BaseCallback
import gym
from gym import spaces
from gym.wrappers import FrameStack
from collections import deque
# Import UnityComms from peaceful_pie.unity_comms
from peaceful_pie.unity_comms import UnityComms
from peaceful_pie.unity_comms import UnityComms
from peaceful_pie import ray_results_helper
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
def unity_comms(port: int):
    unity_comms = UnityComms(port)
    return unity_comms


In [4]:
port = 9000  # Replace with your desired port number
unity_comms_instance = unity_comms(port)

In [5]:
unity_comms_instance 
unity_comms = unity_comms_instance

In [6]:
class MyVector3:
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z

In [7]:
class UnityEnv(Env):
    def __init__(self, unity_comms, i):
        self.unity_comms = unity_comms
        self.action_space = Discrete(15)  # Scale the action range to -1.0 to 1.0
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(3,), dtype=np.float32)
        self.initial_position = None
        self.prev_position = None
        self.prev_velocity = 0
        self.initial_get_CarCollisionDetected =0
        self.frame_count=0
        self.i = i

    def RayCast(self):
        ray_results = getattr(self.unity_comms, f"GetRayCastsResults_{self.i}")()
        distance = np.array(ray_results['rayDistances'], dtype=np.float32)
        types = np.array(ray_results['rayHitObjectTypes'], dtype=np.int32)
        num_types = ray_results['NumObjectTypes']
        self.actual_result = ray_results_helper.ray_results_to_feature_np(
            ray_results_helper.RayResults(
                NumObjectTypes=num_types,
                rayDistances=distance,
                rayHitObjectTypes=types,
            )
        )  

        return self.actual_result 
    

    def Is_obstacle_visible(self):
        actual_result=self.RayCast()
        self.obstacle_channel = actual_result[0]  # Extract the obstacle channel from the actual result
        if np.max(self.obstacle_channel) > 0:
            return True
        return False
    
    def Is_obstacle_Collision_detected(self):
        if self.Is_obstacle_visible()== True:
            actual_result=self.RayCast()
            self.obstacle_channel = actual_result[0]
            if np.max(self.obstacle_channel) >=1:
                return True
            return False

    def Is_reward_visible(self):
        actual_result=self.RayCast()
        reward_channel = actual_result[1]  # Extract the reward channel from the actual result
        if np.max(reward_channel) > 0:
            return True
        return False

    def Goal(self):
        if self.Is_reward_visible() == True:
            goal_channel = self.actual_result[1]
            if np.max(goal_channel) > 1:
                return True
        return False


    def Get_reward(self):
        obstacle_penalty = -0.25  # Penalty for encountering an obstacle
        reward_bonus = .7 # Reward for finding a reward object
        goal_reward = 1  # Reward for reaching the goal
        car_collision_penalty = -1.0  # Penalty for colliding with the car
        valocity_reward = 0.1 #reward for moving forward
        obstacle_collision_panalty = -.5 #penalty for colliding with obstacle

        reward = 0.0

        if self.Is_obstacle_visible()== True:
            reward += obstacle_penalty

        if self.Is_reward_visible()== True:
            reward += reward_bonus

        reward += self.Step_panalty()

        if self.Goal() == True:
            reward += goal_reward

        if self.Is_obstacle_Collision_detected()==True:
            reward +=obstacle_collision_panalty

        if self.Get_carCollisionDetected() == True:
            reward += car_collision_penalty

        if self.Check_valocity_increment()== True:
            reward += valocity_reward

        return reward

    def reset(self):
        self.RayCast()
        Reset_Car = getattr(self.unity_comms, f"ResetPosition_{self.i}")()  
        Reset_Agent = getattr(self.unity_comms, f"ResetPosition_Plane_{self.i}")()
        position =self.Get_position()
        position = np.array(position, dtype=np.float32)
        self.frame_count=0
        return position

    def done(self):
        if self.Get_carCollisionDetected() == True:
            self.frame_count=0
            return True
        elif self.Get_movingPlaneCollision() == True:
            self.frame_count=0
            return True
        elif self.Get_planeCollision() == True:
            self.frame_count=0
            return True
        elif self.Check_stuck() == True:
            self.frame_count=0
            return True
        else:
            self.frame_count += 1
            return False

    def Get_carCollisionDetected(self):
        collision_count = 0
        collision = collision = getattr(self.unity_comms, f"CarCollisionDetected_{self.i}")()
        if collision == 1:
            collision_count += 1
            if collision_count >=2:
                collision_count = 0
                return True

        return False
    
    def Get_movingPlaneCollision(self):
        collision_count = 0
        collision = getattr(self.unity_comms, f"GetMovingPlaneCollision_{self.i}")()
        if collision == 1:
            collision_count += 1
            if collision_count >= 1:
                collision_count = 0
                return True

        return False
    
    def Get_planeCollision(self):
        collision_count = 0
        collision = getattr(self.unity_comms, f"GetPlaneCollision_{self.i}")()
        if collision == 1:
            collision_count += 1
            if collision_count > 0:
                collision_count = 0
                return True

        return False
    
    
    def Get_rewardCollision(self):
        collision = getattr(self.unity_comms, f"GetRewardCollision_{self.i}")()
        self.rewardcollision = collision

    def Step_panalty(self):
        if self.done() == True:
            return -1
        else:
            return 0

    def step(self, action):
        # Perform the action based on the provided action index
        if action == 0:
            getattr(self.unity_comms, f"GoForward_{self.i}")()
        elif action == 1:
            getattr(self.unity_comms, f"GoReverse_{self.i}")()
        elif action == 2:
            getattr(self.unity_comms, f"TurnLeft_{self.i}")()
        elif action == 3:
            getattr(self.unity_comms, f"TurnRight_{self.i}")()
        elif action == 4:
            getattr(self.unity_comms, f"Handbrake_{self.i}")()
        elif action == 5:
            getattr(self.unity_comms, f"GoForward_{self.i}")()
            getattr(self.unity_comms, f"TurnLeft_{self.i}")()
        elif action == 6:
            getattr(self.unity_comms, f"GoForward_{self.i}")()
            getattr(self.unity_comms, f"TurnRight_{self.i}")()
        elif action == 7:
            getattr(self.unity_comms, f"GoForward_{self.i}")()
            getattr(self.unity_comms, f"Handbrake_{self.i}")()
        elif action == 8:
            getattr(self.unity_comms, f"GoReverse_{self.i}")()
            getattr(self.unity_comms, f"TurnLeft_{self.i}")()
        elif action == 9:
            getattr(self.unity_comms, f"GoReverse_{self.i}")()
            getattr(self.unity_comms, f"TurnRight_{self.i}")()
        elif action == 10:
            getattr(self.unity_comms, f"GoReverse_{self.i}")()
            getattr(self.unity_comms, f"Handbrake_{self.i}")()
        elif action == 11:
            getattr(self.unity_comms, f"GoForward_{self.i}")()
            getattr(self.unity_comms, f"TurnLeft_{self.i}")()
            getattr(self.unity_comms, f"Handbrake_{self.i}")()
        elif action == 12:
            getattr(self.unity_comms, f"GoForward_{self.i}")()
            getattr(self.unity_comms, f"TurnRight_{self.i}")()
            getattr(self.unity_comms, f"Handbrake_{self.i}")()
        elif action == 13:
            getattr(self.unity_comms, f"GoReverse_{self.i}")()
            getattr(self.unity_comms, f"TurnLeft_{self.i}")()
            getattr(self.unity_comms, f"Handbrake_{self.i}")()
        elif action == 14:
            getattr(self.unity_comms, f"GoReverse_{self.i}")()
            getattr(self.unity_comms, f"TurnRight_{self.i}")()
            getattr(self.unity_comms, f"Handbrake_{self.i}")()
        else:
            raise ValueError("Invalid action index")

        self.prev_position = self.Get_position()
        self.prev_velocity = self.Get_velocity()

        position = self.Get_position()
        velocity = self.Get_velocity()
        # Concatenate the position and velocity to form the observation
        observation = np.array([position[0], position[1], velocity], dtype=np.float32)

        reward = self.Get_reward()
        done = self.done()
    # Update the info dictionary with relevant information
        info = {
            'episode_reward': reward,  # Replace with the actual episode reward
            'episode_length': self.frame_count,  # Replace with the actual episode length
            'current_observation': observation,
            'action_taken': action,
            'obstacle_visible': self.Is_obstacle_visible(),
            'reward_visible': self.Is_reward_visible(),
            'goal_reached': self.Goal(),
            'car_collision_detected': self.Get_carCollisionDetected(),
            'velocity_incremented': self.Check_valocity_increment(),
        }
        return observation, reward, done, info


    def Get_velocity(self):
        valocity =  getattr(self.unity_comms, f"CarSpeedUI_{self.i}")()
        return valocity

    def Check_valocity_increment(self):
        valocity = self.Get_velocity()
        if valocity > self.prev_velocity:
            self.prev_velocity = valocity
            return True
        else:
            return False

    def Check_stuck(self):
        position_threshold = 0.01
        consecutive_steps = 20
        position_counter = 0
        
        if self.prev_position is None:
            self.prev_position = self.get_position()

        for _ in range(consecutive_steps):
            x, y, z = self.Get_position()

            position_diff = np.linalg.norm(np.array([x, y, z]) - np.array(self.prev_position))
            if position_diff < position_threshold:
                position_counter += 1
            else:
                position_counter = 0

            self.prev_position = [x, y, z]

        if position_counter >= consecutive_steps:
            return True
        else:
            return False


    def Get_position(self):
        position = getattr(self.unity_comms, f"GetPosition_{self.i}")()
        # Extract x, y, and z components from position
        x = position['x']
        y = position['y']
        z = position['z']
        return x, y, z




In [8]:
class Raycast(UnityEnv):
    def __init__(self, unity_comms, i):
        super().__init__(unity_comms, i)

    def Full_ray_analysis(self):
        ray_results = getattr(self.unity_comms, f"GetRayCastsResults_{self.i}")()
        distance = np.array(ray_results['rayDistances'], dtype=np.float32)
        types = np.array(ray_results['rayHitObjectTypes'], dtype=np.int32)
        num_types = ray_results['NumObjectTypes']
        self.actual_result = ray_results_helper.ray_results_to_feature_np(
            ray_results_helper.RayResults(
                NumObjectTypes=num_types,
                rayDistances=distance,
                rayHitObjectTypes=types,
            )
        )  

        return ray_results,distance,types,num_types,self.actual_result 
    
        
        



In [9]:
ray_analysis = Raycast(unity_comms, 9000)

In [10]:
# Define the frame skip frequency
frame_skip_frequency = 5

# Define the number of training steps
total_timesteps = 100000

# Define the directory paths
LOG_DIR = './logs/'
OPT_DIR = './opt_modeldata/'
CHECKPOINT_DIR = './train_modeldata/'

# Create the directories if they don't exist
os.makedirs(LOG_DIR, exist_ok=True)
os.makedirs(OPT_DIR, exist_ok=True)
os.makedirs(CHECKPOINT_DIR, exist_ok=True)


In [11]:
# Import optuna for HPO
import optuna
# Import PPO for algos
from stable_baselines3 import PPO
# Evaluate Policy
from stable_baselines3.common.evaluation import evaluate_policy
# Import wrappers
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack
import os
LOG_DIR = './logs/'
if not os.path.exists(LOG_DIR):
    os.makedirs(LOG_DIR)

OPT_DIR = './opt_modeldata/'
if not os.path.exists(OPT_DIR):
    os.makedirs(OPT_DIR)



c:\Users\rifat\miniconda3\envs\tf\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
total_timesteps = 1000000

In [13]:



# Import os for file path management
import os 
# Import Base Callback for saving models
from stable_baselines3.common.callbacks import BaseCallback


class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True
    
CHECKPOINT_DIR = './train_modeldata/'
if not os.path.exists(CHECKPOINT_DIR):
    os.makedirs(CHECKPOINT_DIR)

callback = TrainAndLoggingCallback(check_freq=10000, save_path=CHECKPOINT_DIR)

env = UnityEnv(unity_comms,8000)
env = Monitor(env, LOG_DIR)
env = DummyVecEnv([lambda: env])
env = VecFrameStack(env, 4, channels_order='last')



callback = TrainAndLoggingCallback(check_freq=10000, save_path=CHECKPOINT_DIR)

from stable_baselines3 import PPO

model_params = {
    'n_steps': 4885,
    'gamma': 0.839729258597705,
    'learning_rate': 4.456287316411457e-05,
    'clip_range': 0.29999077694725984,
    'gae_lambda': 0.9074598206209127
}

model = PPO('MlpPolicy', env, batch_size=4096, tensorboard_log=LOG_DIR, verbose=1, **model_params)


model.load("D:/RL_UNITY/new_env_car_multicar_developer_bug_fixed/Assets/subAssets/Python/environment/Env8000/train_modeldata/best_model_10000.zip")

model.learn(total_timesteps=total_timesteps, callback=callback)






Using cuda device


c:\Users\rifat\miniconda3\envs\tf\lib\site-packages\stable_baselines3\ppo\ppo.py:140: UserWarning: You have specified a mini-batch size of 4096, but because the `RolloutBuffer` is of size `n_steps * n_envs = 4885`, after every 1 untruncated mini-batches, there will be a truncated mini-batch of size 789
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=4885 and n_envs=1)
  warnings.warn(


Logging to ./logs/PPO_5
requests.exceptions.ConnectionError => ignoring, retrying
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 4.48     |
|    ep_rew_mean     | 0.951    |
| time/              |          |
|    fps             | 0        |
|    iterations      | 1        |
|    time_elapsed    | 6532     |
|    total_timesteps | 4885     |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 2.82         |
|    ep_rew_mean          | -1.1         |
| time/                   |              |
|    fps                  | 0            |
|    iterations           | 2            |
|    time_elapsed         | 13261        |
|    total_timesteps      | 9770         |
| train/                  |              |
|    approx_kl            | 8.099606e-05 |
|    clip_fraction        | 0            |
|    clip_range           | 0.3          |
|    entropy_loss  